<h1>Naive Bayes</h1>

In [1]:
library(mlr)
source('../utils.r')

set.seed(42)

folder_name = '../../raw_data' 
file_name   = 'german.csv'

Loading required package: ParamHelpers
Warning message:
“replacing previous import ‘BBmisc::isFALSE’ by ‘backports::isFALSE’ when loading ‘mlr’”

<h2>1. Dataprep</h2>

In [2]:
data = read.csv(file=sprintf('%s/%s',folder_name,file_name))

In [3]:
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))
head(data)

NRow: 1000
NCol: 22

X,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
1,A11,6,A34,A43,1169,A65,A75,4,A93,⋯,A121,67,A143,A152,2,A173,good,A192,A201,1
2,A12,48,A32,A43,5951,A61,A73,2,A92,⋯,A121,22,A143,A152,1,A173,good,A191,A201,2
3,A14,12,A34,A46,2096,A61,A74,2,A93,⋯,A121,49,A143,A152,1,A172,bad,A191,A201,1
4,A11,42,A32,A42,7882,A61,A74,2,A93,⋯,A122,45,A143,A153,1,A173,bad,A191,A201,1
5,A11,24,A33,A40,4870,A61,A73,3,A93,⋯,A124,53,A143,A153,2,A173,bad,A191,A201,2
6,A14,36,A32,A46,9055,A65,A73,2,A93,⋯,A124,35,A143,A153,1,A172,bad,A192,A201,1


<p style='color: red'> ATENTION: </p>Specifically in R, in classification problem, target must be set as Factor.

In [4]:
data$V21 = as.factor(data$V21)

MLR works only with features and target, this means that others columns must be dorped.

In [5]:
drops = c('X')
data  = data[ , !(names(data) %in% drops)]
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 21

<h2>2. Modeling</h2>

Function
```R
makeLearner(cl, id = cl, predict.type = "response", predict.threshold = NULL, 
            fix.factors.prediction = FALSE, ..., par.vals = list(), config = list())
```
Param.:

* cl: [character(1)] Class of learner. By convention, all classification learners start with “classif.”. A list of all integrated learners is available on the learners help page < https://mlr-org.github.io/mlr-tutorial/release/html/integrated_learners/ >.
* predict: [character(1)] “response” (= labels) or “prob” (= probabilities and labels by selecting the ones with maximal probability). Default is “response”.
* par.vals: [list] Optional list of named (hyper)parameters. The arguments in ... take precedence over values in this list. We strongly encourage you to use one or the other to pass (hyper)parameters to the learner but not both.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [6]:
cl = "classif.naiveBayes"

List all parameters that can be used in this classifier. The value must be set in par.vals parameter.

In [7]:
getParamSet(cl)

           Type len Def   Constr Req Tunable Trafo
laplace numeric   -   0 0 to Inf   -    TRUE     -

In [8]:
learner = makeLearner(cl = cl
                     , predict.type = "prob"
                     , par.vals = list()
                     )

Function
```R
makeClassifTask(id = deparse(substitute(data)), data, target, weights = NULL, blocking = NULL, 
                positive = NA_character_, fixup.data = "warn", check.data = TRUE)
```
Param.:

* data: [data.frame] A data frame containing the features and target variable(s).
* target: [character(1)] Name of the target variable.
* positive: [character(1)] Positive class for binary classification (otherwise ignored and set to NA). Default is the first factor level of the target attribute.
* fixup.data: [character(1)] Should some basic cleaning up of data be performed? Currently this means removing empty factor levels for the columns. Possible coices are: “no” = Don't do it. “warn” = Do it but warn about it. “quiet” = Do it but keep silent. Default is “warn”.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [9]:
task = makeClassifTask( data = data
                      , target = 'V21'
                      , positive = '2'
                      , fixup.data = 'no'
                      )

Function:
```R
makeResampleDesc(method, predict = "test", ..., stratify = FALSE, stratify.cols = NULL)
```
Param.:

* method: [character(1)] “CV” for cross-validation, “LOO” for leave-one-out, “RepCV” for repeated cross-validation, “Bootstrap” for out-of-bag bootstrap, “Subsample” for subsampling, “Holdout” for holdout.
* predict: What to predict during resampling: “train”, “test” or “both” sets. Default is “test”.
* ... : [any] Further parameters for strategies.
    * iters [integer(1)] Number of iterations, for “CV”, “Subsample” and “Boostrap”.
    * split [numeric(1)] Proportion of training cases for “Holdout” and “Subsample” between 0 and 1. Default is 2/3.
    * reps [integer(1)] Repeats for “RepCV”. Here iters = folds * reps. Default is 10.
    * folds [integer(1)] Folds in the repeated CV for RepCV. Here iters = folds * reps. Default is 10.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeResampleDesc

In [10]:
resample = makeResampleDesc( method = "CV"
                           , iters = 10
                           , predict = 'both'
                           , stratify = FALSE
                           )

List of performance measures:

Doc.: http://mlr-org.github.io/mlr-tutorial/release/html/measures/

In [11]:
measures = list(mmce #MMCE 
               ,acc  #acuracia
               ,f1   #f1
               ,ppv  #precision
               ,tpr  #recall
               ,auc  #AUC
               ,gini #Gini
               ,timetrain #tempo execucao
               )

Function:
```R
resample(learner, task, resampling, measures, weights = NULL, models = FALSE, extract, 
         keep.pred = TRUE, ..., show.info = getMlrOption("show.info"))
```
Param.:

* learner: [Learner] The learner.
* task: [Task] The task.
* resampling: [ResampleInstance] Resampling strategy.
* measures: [Measure | list of Measure] Performance measure(s) to evaluate. Default is mean misclassification error (mmce)
* weights: [numeric] Optional, non-negative case weight vector to be used during fitting. If given, must be of same length as observations in task and in corresponding order. Overwrites weights specified in the task. By default NULL which means no weights are used unless specified in the task.
* models: [logical(1)] Should all fitted models be returned? Default is FALSE.
* keep.pred: [logical(1)] Keep the prediction data in the pred slot of the result object. If you do many experiments (on larger data sets) these objects might unnecessarily increase object size / mem usage, if you do not really need them. In this case you can set this argument to FALSE. Default is TRUE.
* show.info: [logical(1)] Print verbose output on console? Default is set via configureMlr.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/resample

In [12]:
r = resample(learner = learner
            ,task = task 
            ,resampling = resample 
            ,measures = measures
            #---------------------#
            ,models = TRUE
            ,keep.pred = FALSE
            ,show.info = TRUE)

[Resample] cross-validation iter 1: mmce.test.mean=0.28,acc.test.mean=0.72,f1.test.mean=0.462,ppv.test.mean=0.48,tpr.test.mean=0.444,auc.test.mean=0.747,gini.test.mean=0.494,timetrain.test.mean=0.009
[Resample] cross-validation iter 2: mmce.test.mean= 0.3,acc.test.mean= 0.7,f1.test.mean=0.464,ppv.test.mean=0.565,tpr.test.mean=0.394,auc.test.mean=0.751,gini.test.mean=0.502,timetrain.test.mean=0.05
[Resample] cross-validation iter 3: mmce.test.mean=0.28,acc.test.mean=0.72,f1.test.mean=0.517,ppv.test.mean= 0.6,tpr.test.mean=0.455,auc.test.mean=0.801,gini.test.mean=0.602,timetrain.test.mean=0.014
[Resample] cross-validation iter 4: mmce.test.mean= 0.2,acc.test.mean= 0.8,f1.test.mean=0.63,ppv.test.mean=0.773,tpr.test.mean=0.531,auc.test.mean=0.811,gini.test.mean=0.621,timetrain.test.mean=0.007
[Resample] cross-validation iter 5: mmce.test.mean=0.27,acc.test.mean=0.73,f1.test.mean=0.509,ppv.test.mean=0.56,tpr.test.mean=0.467,auc.test.mean=0.757,gini.test.mean=0.513,timetrain.test.mean=0.008


<h2>3. Result Analysis</h2>

Train Measures

In [13]:
r$measures.train

iter,mmce,acc,f1,ppv,tpr,auc,gini,timetrain
1,0.2266667,0.7733333,0.5984252,0.6468085,0.5567766,0.8165752,0.6331505,0.009
2,0.2233333,0.7766667,0.5906314,0.6473214,0.5430712,0.8131956,0.6263912,0.050
3,0.2255556,0.7744444,0.5882353,0.6415929,0.5430712,0.8152783,0.6305566,0.014
4,0.2388889,0.7611111,0.5708583,0.6137339,0.5335821,0.8066196,0.6132392,0.007
5,0.2288889,0.7711111,0.5912698,0.6367521,0.5518519,0.8099588,0.6199177,0.008
6,0.2266667,0.7733333,0.5853659,0.6457399,0.5353160,0.8141205,0.6282410,0.013
7,0.2322222,0.7677778,0.5708419,0.6318182,0.5205993,0.8058943,0.6117886,0.006
8,0.2322222,0.7677778,0.5861386,0.6379310,0.5421245,0.8123864,0.6247729,0.007
9,0.2311111,0.7688889,0.5937500,0.6413502,0.5527273,0.8113105,0.6226211,0.006
10,0.2266667,0.7733333,0.5920000,0.6462882,0.5461255,0.8105762,0.6211523,0.005


Test Measures

In [14]:
r$measures.test

iter,mmce,acc,f1,ppv,tpr,auc,gini,timetrain
1,0.28,0.72,0.4615385,0.4800000,0.4444444,0.7468290,0.4936580,0.009
2,0.30,0.70,0.4642857,0.5652174,0.3939394,0.7512438,0.5024876,0.050
3,0.28,0.72,0.5172414,0.6000000,0.4545455,0.8009950,0.6019900,0.014
4,0.20,0.80,0.6296296,0.7727273,0.5312500,0.8106618,0.6213235,0.007
5,0.27,0.73,0.5090909,0.5600000,0.4666667,0.7566667,0.5133333,0.008
6,0.27,0.73,0.5090909,0.5833333,0.4516129,0.7587658,0.5175316,0.013
7,0.18,0.82,0.7352941,0.7142857,0.7575758,0.8421529,0.6843057,0.006
8,0.23,0.77,0.5490196,0.5833333,0.5185185,0.7980720,0.5961441,0.007
9,0.17,0.83,0.5853659,0.7500000,0.4800000,0.8176000,0.6352000,0.006
10,0.27,0.73,0.5263158,0.5357143,0.5172414,0.7785333,0.5570665,0.005


Train Aggregated Result

In [15]:
apply(r$measures.train,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.2292222 0.7707778 0.5867516 0.6389336 0.5425245 0.8115915 0.6231831 
timetrain 
0.0125000

Test Aggregated Result

In [16]:
apply(r$measures.test,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.2450000 0.7550000 0.5486872 0.6144611 0.5015795 0.7861520 0.5723040 
timetrain 
0.0125000

Run Time in seconds

In [17]:
r$runtime

[1] 5.147549

<h2>4. Prediction for new data</h2>

Read the data to predict

In [18]:
new.data = read.csv(file=sprintf('%s/%s', folder_name, file_name))

Search for the best model in crossvalidation and use it to score the incoming data

In [19]:
best.model = which.max(r$measures.test$acc)

In [20]:
pred = predict(r$models[[best.model]], newdata = new.data)

Prediction Result

In [21]:
pred

Prediction: 1000 observations
predict.type: prob
threshold: 1=0.50,2=0.50
time: 0.32
  truth    prob.1     prob.2 response
1     1 0.9875594 0.01244062        1
2     2 0.3090151 0.69098487        2
3     1 0.9866216 0.01337840        1
4     1 0.1814895 0.81851051        2
5     2 0.1872980 0.81270202        2
6     1 0.2988458 0.70115419        2
... (1000 rows, 4 cols)


Cast result to data.frame to access the prediction

In [22]:
head(as.data.frame(pred))

truth,prob.1,prob.2,response
1,0.9875594,0.01244062,1
2,0.3090151,0.69098487,2
1,0.9866216,0.01337840,1
1,0.1814895,0.81851051,2
2,0.1872980,0.81270202,2
1,0.2988458,0.70115419,2
